In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os, sys, itertools


module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from util.operon import get_operon_name_set, get_operon_ID_set
from util.genome import NON_K12_EXP_L
from util.gene import get_gene_bnum
from util.params import ENRICH_ALPHA, MULTI_HYP_CORR_METHOD
pd.options.display.max_columns = 100

In [2]:
# exp_muts_df = pd.read_pickle("./data/9_df.pkl")  # Currently skipping imodulons for the sake of the AVA MS
exp_muts_df = pd.read_pickle("./data/8_df.pkl")

display(exp_muts_df.shape, exp_muts_df.head())

(46, 41)

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1)


In [3]:
# # debug
# exp_muts_fd = pd.read_pickle("./data/2_4_df.pkl")
# exp_muts_fd = exp_muts_fd[exp_muts_fd.exp=="SER"]
# display(exp_muts_fd.shape, exp_muts_fd.head())

In [4]:
cog_df = pd.read_csv("./data/cogs_ecoli_mg1655.csv", encoding = "ISO-8859-1")
cog_df = cog_df.drop(["GI", "COG ID", "COG name", "COG category"], axis=1).drop_duplicates()
cog_df = cog_df[~(cog_df["COG description"]=="Function unknown")]  # We want to ignore the "Function unknown" COG entries because we ignore it and it's mutation for the statistical work and mutflow visualizations
display(len(cog_df), cog_df.head())

3441

,gene,locus,COG category primary,COG class,COG description
0,thrA,b0002,E,METABOLISM,Amino acid transport and metabolism
1,metL,b3940,E,METABOLISM,Amino acid transport and metabolism
4,lysC,b4024,E,METABOLISM,Amino acid transport and metabolism
5,thrB,b0003,E,METABOLISM,Amino acid transport and metabolism
6,thrC,b0004,E,METABOLISM,Amino acid transport and metabolism


In [5]:
TU_objects_df = pd.read_csv("./data/RegulonDB10/tu_objects_tmp.txt",
                            sep="\t",
                            comment='#',
                            header=None,
                            )

TU_objects_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "NUMTU",
    "TU_POSLEFT",
    "TU_POSRIGHT",
    "TU_TYPE",
    "TU_OBJECT_CLASS",
    "TU_OBJECT_ID",
    "TU_OBJECT_NAME",
    "TU_OBJECT_POSLEFT",
    "TU_OBJECT_POSRIGHT",
    "TU_OBJECT_STRAND",
    "TU_OBJECT_COLORCLASS",
    "TU_OBJECT_DESCRIPTION",
    "TU_OBJECT_SIGMA",
    "TU_OBJECT_EVIDENCE",
    "TU_OBJECT_RI_TYPE",
    "TU_OBJECT_TYPE",
    "EVIDENCE"]


operon_df = pd.read_csv("./data/RegulonDB10/operon.txt", sep="\t", comment='#', header=None)
operon_df.columns = [
    "OPERON_ID",
    "OPERON_NAME",
    "FIRSTGENEPOSLEFT",
    "LASTGENEPOSRIGHT",
    "REGULATIONPOSLEFT",
    "REGULATIONPOSRIGHT",
    "OPERON_STRAND",
    "OPERON_INTERNAL_COMMENT",
    "KEY_ID_ORG"]
operon_df["range"] = operon_df.apply(lambda row: (row["REGULATIONPOSLEFT"], row["REGULATIONPOSRIGHT"]), axis=1)


TU_df = pd.read_csv("./data/RegulonDB10/transcription_unit.txt", sep="\t", comment='#', header=None)
TU_df.columns = [
    "TRANSCRIPTION_UNIT_ID",
    "PROMOTER_ID",
    "TRANSCRIPTION_UNIT_NAME",
    "OPERON_ID",
    "TRANSCRIPTION_UNIT_NOTE",
    "TU_INTERNAL_COMMENT",
    "KEY_ID_ORG"]

# Get COGs mapping to genetic features affected by mutations

!!! Currently using lists instead of sets for COG hits because I want to know the count of items per COG affected with each mutation, where a mutation can affect multiple unique genomic targets that belong to the same COG. If I used a set, would lose the magnitude of hits to a COG per mutations.  
THIS SHOULD GET FIXED WITH COG LINK DICTS, WHERE THE NUMBER OF TIMES AN OPERON SHOWS UP CAN IN THE LINKING LIST CAN BE THE NUMBER OF GENES INVOVLED IN MUTATING THE COG.

## Get coding mutation COGs

In [6]:
gene_synonym_df = pd.read_csv(
    "./data/RegulonDB10/object_synonym.txt",
    sep="\t",
    comment='#',
    header=None,
    quoting=3
)
gene_synonym_df.columns = ["OBJECT_ID", "OBJECT_SYNONYM_NAME", "OS_INTERNAL_COMMENT", "KEY_ID_ORG"]
gene_synonym_df.head()

,OBJECT_ID,OBJECT_SYNONYM_NAME,OS_INTERNAL_COMMENT,KEY_ID_ORG
0,ECK120000001,EG10001,NaN,ECK12
1,ECK120000001,ECK4045,NaN,ECK12
2,ECK120000001,b4053,NaN,ECK12
3,ECK120000001,alr5,NaN,ECK12
4,ECK120000002,b0764,NaN,ECK12


In [7]:
# This is currently not attributing gene without COG to "Function unknown"
# Last time checking this, it wasn't necessary to add the addition "Function unknown" entries here
# though these extra entries were required with the link dictionaries
def _get_COGs(gene_id):
    annots = []
    bnum = get_gene_bnum(gene_id, gene_synonym_df)
    gene_cog_df = cog_df[cog_df["locus"] == bnum]
    if len(gene_cog_df) > 0:
        for COG in gene_cog_df["COG description"]:
            d = {"name": COG}
            annots.append(d)
    return annots


def get_COGs(mut_row):
    annots = []
    if mut_row["exp"] not in NON_K12_EXP_L:
        for feat_d in mut_row["genomic features"]:
            if feat_d["genetic"]:
                annots += _get_COGs(feat_d["RegulonDB ID"])
            # This will catch whether no features exist in the feat_ID_set
            # or if those features are just integenic regions.
            # Get operon genetic feature link dict through overlapping mutation and operon ranges.
            # All genetic features are annotated within both the "genetic features" and "genetic feature links" column.
            # If no integenic genomic feature is hit by a mutation, use the genetic feature range in an overlap analysis with operons.
            else:
                TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"] == feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
                for TU_ID in TU_IDs:
                    subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"] == TU_ID) & (
                        TU_objects_df["TU_OBJECT_CLASS"] == 'GN')]
                    for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                        # RegulonDB can return 'nan' or '' gene names
                        if str(gene_id) != 'nan' and gene_id != '':
                            annots += _get_COGs(gene_id)
    return annots


test_mut_row_d = {
    "exp": "test exp",
    "genomic features":
    [
        {'name': 'rph-pyrE attenuator terminator',
         'RegulonDB ID': 'ECK125144791',
         'range': (3815799, 3815828),
         'genetic': False,
         'feature type': 'attenuator terminator',
         'operon': 'rph-pyrE',
         'mutation set count': 5,
         'significant': True},
        {'name': 'rph',
         'RegulonDB ID': 'ECK120000854',
         'range': (3815863, 3816549),
         'genetic': True,
         'feature type': 'gene',
         'operon': 'rph-pyrE',
         'mutation set count': 5,
         'significant': True}
    ]}
assert(get_COGs(test_mut_row_d) == [{'name': 'Nucleotide transport and metabolism'}])

In [8]:
exp_muts_df["COGs"] = exp_muts_df.apply(lambda r: get_COGs(r), axis=1)
exp_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source,COGs
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Transcription'}]
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism'}]
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Transcription'}]
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Replication, recombination and repa..."
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism'}]


In [9]:
def _add_links_to_d(bnum, d, op_ID_set):
    gene_cog_df = cog_df[cog_df["locus"]==bnum]
    if len(gene_cog_df) > 0:
        for cog in gene_cog_df["COG description"]:
            if cog not in d.keys(): d[cog] = []
            d[cog] += list(op_ID_set)
#     else:
#         if "Function unknown" not in d.keys(): d["Function unknown"] = []
#         d["Function unknown"] = list(op_ID_set)


def get_COG_operon_links(mut_row):
    COG_op_link_d = dict()
    if mut_row.exp not in NON_K12_EXP_L:
        for feat_d in mut_row["genomic features"]:
            
#             link_exists = False
            
            if feat_d["genetic"]:
                op_ID_set = get_operon_ID_set(feat_d["RegulonDB ID"], TU_objects_df, TU_df, operon_df)
                if str(feat_d["RegulonDB ID"]) != 'nan' and feat_d["RegulonDB ID"] != '':
                    bnum = get_gene_bnum(feat_d["RegulonDB ID"], gene_synonym_df)
                    _add_links_to_d(bnum, COG_op_link_d, op_ID_set)
#                     link_exists = True
            else:
                TU_IDs = set(TU_objects_df[TU_objects_df["TU_OBJECT_ID"]==feat_d["RegulonDB ID"]]["TRANSCRIPTION_UNIT_ID"])
                for TU_ID in TU_IDs:
                    subset_TU_genes_df = TU_objects_df[(TU_objects_df["TRANSCRIPTION_UNIT_ID"]==TU_ID) & (TU_objects_df["TU_OBJECT_CLASS"]=='GN')]

                    op_ID_set = set()
                    for _, TU_gene in subset_TU_genes_df.iterrows():
                         op_ID_set = op_ID_set | get_operon_ID_set(TU_gene["TU_OBJECT_ID"], TU_objects_df, TU_df, operon_df)

                    for gene_id in subset_TU_genes_df["TU_OBJECT_ID"].drop_duplicates():
                        if str(gene_id) != 'nan' and gene_id != '':  # RegulonDB can return 'nan' or '' gene names
                            bnum = get_gene_bnum(gene_id, gene_synonym_df)
                            _add_links_to_d(bnum, COG_op_link_d, op_ID_set)
#                             link_exists = True
                            
            # Per mutation feature
            # Need to be looking at previously populated level of annotations depdending on how the current level is being populated
            # for regulon, it just means that if there are no operons, go immediately to genomic features.
#             if link_exists == False:
#                 if "Function unknown" not in COG_op_link_d.keys(): COG_op_link_d["Function unknown"] = []
#                 if len(operon_name_set) > 0: # need to always check the nearest annotation type for defined features
#                     COG_op_link_d["Function unknown"] += list(operon_name_set)
#                 else:
#                     COG_op_link_d["Function unknown"].append(feat_d["RegulonDB ID"])
                        
    return COG_op_link_d


# get_COG_operon_links(exp_muts_df.loc[2104])

In [10]:
exp_muts_df["COG links"] = exp_muts_df.apply(get_COG_operon_links, axis=1)
exp_muts_df.head()

,index,Details,mutation target annotation,Mutation Type,Position,Reference Seq,Sequence Change,ale,exp,flask,isolate,presence,tech_rep,coding,range,gene RegulonDB ID,genetic features,oriC,pseudogene,TFBS,promoter,RBS,attenuator terminator,terminator,genetic,genomic features,genetic feature links,operons,operon links,regulators,regulator links,temperature,carbon-source,supplement,strain-description,taxonomy-id,base-media,nitrogen-source,phosphorous-source,sulfur-source,calcium-source,COGs,COG links
1232,4,R121C (CGC→TGC),xylR,SNP,3735339,NaN,C→T,1,SSW_XYL,116,1,1.0,1,True,"(3735339, 3735339)",{ECK120002449},"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",False,False,{},{},{},{},{},True,"[{'name': 'xylR', 'RegulonDB ID': 'ECK12000244...",{'ECK120002449': ['ECK120002449']},"[{'name': 'xylFGHR', 'RegulonDB ID': 'ECK12001...",{'ECK120014482': ['ECK120002449']},"[{'name': 'Fis', 'significantly associated con...","{'ArcA': ['xylFGHR'], 'Fis': ['xylFGHR'], 'CRP...",37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Transcription'}],{'Transcription': ['ECK120014482']}
1233,5,None,rph,DEL,3815859,NaN,Δ82 bp,1,SSW_XYL,116,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...
1234,6,L770R (CTC→CGC),rpoC,SNP,4187658,NaN,T→G,1,SSW_XYL,116,1,1.0,1,True,"(4187658, 4187658)",{ECK120000886},"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",False,False,{},{},{},{},{},True,"[{'name': 'rpoC', 'RegulonDB ID': 'ECK12000088...",{'ECK120000886': ['ECK120000886']},"[{'name': 'rplKAJL-rpoBC', 'RegulonDB ID': 'EC...",{'ECK120016992': ['ECK120000886']},[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Transcription'}],{'Transcription': ['ECK120016992']}
1235,9,G164A (GGC→GCC),dnaG,SNP,3211597,NaN,G→C,2,SSW_XYL,109,1,1.0,1,True,"(3211597, 3211597)",{ECK120000233},"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",False,False,{},{},{},{},{},True,"[{'name': 'dnaG', 'RegulonDB ID': 'ECK12000023...",{'ECK120000233': ['ECK120000233']},"[{'name': 'rpsU-dnaG-rpoD', 'RegulonDB ID': 'E...",{'ECK120014515': ['ECK120000233']},"[{'name': 'LexA', 'significantly associated co...",{'LexA': ['rpsU-dnaG-rpoD']},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),"[{'name': 'Replication, recombination and repa...","{'Replication, recombination and repair': ['EC..."
1236,5,None,rph,DEL,3815859,NaN,Δ82 bp,2,SSW_XYL,109,1,1.0,1,True,"(3815859, 3815940)",{ECK120000854},"[{'name': 'rph', 'RegulonDB ID': 'ECK120000854...",False,False,{},{},{},{ECK125144791},{},True,"[{'name': 'rph-pyrE attenuator terminator', 'R...","{'ECK120000854': ['ECK125144791', 'ECK12000085...","[{'name': 'rph-pyrE', 'RegulonDB ID': 'ECK1200...","{'ECK120014627': ['ECK120000854', 'ECK12000085...",[],{},37 celsius,xylose(4),NaCl(0.5g/L) trace elements,WT,511145,M9,NH4Cl(1),KH2PO4(3) Na2HPO4(6.8),MgSO4(0.24),CaCl2(0.1),[{'name': 'Nucleotide transport and metabolism'}],{'Nucleotide transport and metabolism': ['ECK1...


# significance

In [11]:
# a bit behind annotations currently used in AVA MS.
# Still uses same approach to counted, just not all data-types have been consolidated into the COGs feat col


comp_mut_df = pd.DataFrame(columns=["observed mutation count"])
for i, r in exp_muts_df.iterrows():
    for f, links in r["COG links"].items():
        if "unknown" not in f:
            if f not in comp_mut_df.index:
                df = pd.DataFrame({"observed mutation count": len(links)}, index=[f])
                comp_mut_df = comp_mut_df.append(df)
            else:
                comp_mut_df.loc[f, "observed mutation count"] += len(links)
            
comp_mut_df

,observed mutation count
Transcription,10
Nucleotide transport and metabolism,5
"Replication, recombination and repair",2
Cell motility,7
Extracellular structures,1
Amino acid transport and metabolism,2
Inorganic ion transport and metabolism,2
"Posttranslational modification, protein turnover, chaperones",1
Cell wall/membrane/envelope biogenesis,3
Carbohydrate transport and metabolism,4


In [12]:
np.sum(comp_mut_df["observed mutation count"])

44

In [13]:
for _, r in exp_muts_df.iterrows():
    for feat_d in r["COGs"]:
        if "unknown" not in feat_d["name"]:
            feat_d["mutation set count"] = comp_mut_df.loc[feat_d["name"]]["observed mutation count"]

In [14]:
COG_len_df = pd.read_pickle("./data/COG_len_df.pkl")
COG_len_df.head()

,COG length
COG description,
Amino acid transport and metabolism,660007
Inorganic ion transport and metabolism,305849
Carbohydrate transport and metabolism,679809
Coenzyme transport and metabolism,212728
Energy production and conversion,589566


In [15]:
import numpy as np


comp_mut_df["length"] = comp_mut_df.apply(lambda r: COG_len_df.loc[r.index, "COG length"])
total_seq_len = np.sum(comp_mut_df["length"])
comp_mut_df["proportion"] = comp_mut_df["length"].apply(lambda comp_len: comp_len/total_seq_len)
comp_mut_df

,observed mutation count,length,proportion
Transcription,10,424603,0.090876
Nucleotide transport and metabolism,5,160339,0.034316
"Replication, recombination and repair",2,309957,0.066338
Cell motility,7,144533,0.030934
Extracellular structures,1,12182,0.002607
Amino acid transport and metabolism,2,660007,0.141258
Inorganic ion transport and metabolism,2,305849,0.065459
"Posttranslational modification, protein turnover, chaperones",1,270912,0.057982
Cell wall/membrane/envelope biogenesis,3,520583,0.111418
Carbohydrate transport and metabolism,4,679809,0.145496


In [16]:
from collections import Counter
import numpy as np
comp_mut_df['equal or larger count'] = 0
component_l = comp_mut_df.index.tolist()
proportion_l = comp_mut_df["proportion"].tolist()
num_muts = np.sum(comp_mut_df["observed mutation count"])

# slides from UW speak about the relationship of permutations and precision.
# should read through them to get a sense of what is the least iterations that should execute.
num_itr = 10000
for i in range(0, num_itr):
    random_mut_comp_a = np.random.choice(
        a=component_l,
        size=int(num_muts),
        p=proportion_l,
        replace=True,
    )
    random_mut_comp_count_d = Counter(random_mut_comp_a)
    for component in random_mut_comp_count_d.keys():
        if random_mut_comp_count_d[component] >= comp_mut_df.loc[component, 'observed mutation count']:
            comp_mut_df.loc[component, 'equal or larger count'] += 1

comp_mut_df["p value"] = comp_mut_df["equal or larger count"].apply(
    lambda count: count/num_itr)

comp_mut_df["significant"] = comp_mut_df["p value"].apply(lambda p_val: True if p_val < ENRICH_ALPHA else False)

display(comp_mut_df.head())

,observed mutation count,length,proportion,equal or larger count,p value,significant
Transcription,10,424603,0.090876,51,0.0051,True
Nucleotide transport and metabolism,5,160339,0.034316,172,0.0172,True
"Replication, recombination and repair",2,309957,0.066338,8105,0.8105,False
Cell motility,7,144533,0.030934,6,0.0006,True
Extracellular structures,1,12182,0.002607,1054,0.1054,False


In [17]:
from statsmodels.stats import multitest
pvals = comp_mut_df["p value"]
rejects, pvals_corrected, alphacSidak, alphacBonf = multitest.multipletests(
    pvals=pvals,
    alpha=ENRICH_ALPHA,
    method=MULTI_HYP_CORR_METHOD)

pd.options.display.max_rows = 500
comp_mut_df["corrected p value"] = pvals_corrected
comp_mut_df["corrected significance"] = rejects
comp_mut_df.head()

,observed mutation count,length,proportion,equal or larger count,p value,significant,corrected p value,corrected significance
Transcription,10,424603,0.090876,51,0.0051,True,0.0663,False
Nucleotide transport and metabolism,5,160339,0.034316,172,0.0172,True,0.2236,False
"Replication, recombination and repair",2,309957,0.066338,8105,0.8105,False,1.0000,False
Cell motility,7,144533,0.030934,6,0.0006,True,0.0078,True
Extracellular structures,1,12182,0.002607,1054,0.1054,False,1.0000,False


In [18]:
# # not wanting to consider mutation events with only 1 features as being significant
# comp_mut_df["final significance"] = comp_mut_df.apply(lambda r: r["corrected significance"] if r["observed mutation count"] > 1 else False, axis=1)
# comp_mut_df.head()

In [19]:
for i, r in exp_muts_df.iterrows():
    for d in r["COGs"]:
        if d["name"] in comp_mut_df.index:
            d["significantly enriched"] = comp_mut_df.loc[d["name"]]["corrected significance"]

In [20]:
exp_muts_df.to_pickle("./data/10_df.pkl")

In [21]:
# import seaborn as sns
# import matplotlib
# import matplotlib.pyplot as plt
# %matplotlib inline
# plt.rcParams["figure.dpi"] = 100
# sns.set_context("talk")
# sns.set_style("ticks")


# def _get_name_signif_from_name(name, json):
#     is_signif = False
#     for d in json:
#         if d["name"] == name:
#             is_signif = d["significant"]
#             break
#     return is_signif


# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#         if t not in targ_cnt_d.keys():
#             targ_cnt_d[t] = 0
#         targ_cnt_d[t] += len(hits)
        
# df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
# df = df.sort_values("mutated feature count")
# df = df[-20:]
# ax = df.plot.barh(y='mutated feature count',
# #                   figsize=(10,25),
#                   figsize=(5,10),
#                   color="#4C72B0",
#                   width=1
#                  )
# for i, v in enumerate(df["mutated feature count"]):
#     ax.text(v, i - 0.2, str(v))
# sns.despine(ax=ax, top=True, right=True)
# ax.set_title("Top 20 mutated COGs")
# ax.get_legend().remove()

In [22]:
# # The reason we're using the links dictionary is because we want to count how genomic features were mutated for the genetic target.
# targ_cnt_d = dict()
# for _, r in exp_muts_df.iterrows():
#     link_d = r["COG links"]
#     for targ_name, hits in link_d.items():
#         is_signif = _get_name_signif_from_name(targ_name, r["COGs"])
#         t = str(targ_name)
#         if is_signif:
#             t = '*' + t
#             if t not in targ_cnt_d.keys():
#                 targ_cnt_d[t] = 0
#             targ_cnt_d[t] += len(hits)
# if len(targ_cnt_d) > 0:
#     df = pd.DataFrame.from_dict(targ_cnt_d, orient='index', columns=["mutated feature count"])
#     df = df.sort_values("mutated feature count")
#     df = df[-20:]
#     ax = df.plot.barh(y='mutated feature count',
#     #                   figsize=(10,25),
#     #                   figsize=(5,10),
#                       color="#4C72B0",
#                       width=1
#                      )
#     for i, v in enumerate(df["mutated feature count"]):
#         ax.text(v, i - 0.2, str(v))
#     sns.despine(ax=ax, top=True, right=True)
#     ax.set_title("Signifcant mutated COGs")
#     ax.get_legend().remove()